In [1]:
import pandas as pd
import numpy as np
import re
import random

In [2]:
df1 = pd.read_excel (r'Fall2021_Perceptions_Explanations_All_Final.xlsx')
df2 = pd.read_excel (r'Spring2022_Perceptions_Explanations_All_Final.xlsx')
df = pd.concat([df1,df2])
df = df[df["misplaced"].isnull()]
#df = df[df["explanation"].notnull()]
#df = df[df["explanation"].isstr()]

In [3]:
len(df)

1107

In [3]:
def define_code_col(df_dum,code):
    df_dum[code] = np.where(df_dum["code1"] == code,1,0) + np.where(df_dum["code2"] == code,1,0) + np.where(df_dum["code3"] == code,1,0)+ np.where(df_dum["code4"] == code,1,0)+ np.where(df_dum["code5"] == code,1,0)
    return df_dum


In [4]:
df = define_code_col(df,"CON")
df = define_code_col(df,"EXT")
df = define_code_col(df,"PER")
df = define_code_col(df,"NAT")
df = define_code_col(df,"MOT")
df = define_code_col(df,"ANA")
df = define_code_col(df,"PROB")
df = define_code_col(df,"PLAN")
df = define_code_col(df,"EXEC")
df = define_code_col(df,"WRIT")
df = define_code_col(df,"PART")
df = define_code_col(df,"LEAD")
df = define_code_col(df,"EXP")
df = define_code_col(df,"HELP")
df = define_code_col(df,"OTH")

In [5]:
df

,context,explanation,misplaced,code1,code2,code3,code4,code5,CON,EXT,...,ANA,PROB,PLAN,EXEC,WRIT,PART,LEAD,EXP,HELP,OTH
0,lab,He has a great grasp of all of the different q...,NaN,CON,ANA,NaN,NaN,NaN,1,0,...,1,0,0,0,0,0,0,0,0,0
1,lab,Their concepts are very strong and they always...,NaN,CON,PLAN,NaN,NaN,NaN,1,0,...,0,0,1,0,0,0,0,0,0,0
2,lab,Focused on his work.,NaN,MOT,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,lab,Has great understanding of software to help an...,NaN,ANA,NaN,NaN,NaN,NaN,0,0,...,1,0,0,0,0,0,0,0,0,0
4,lab,Works at air and space museum,NaN,EXT,NaN,NaN,NaN,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,lecture,Strong grasp on the material and knows how to ...,NaN,CON,PROB,NaN,NaN,NaN,1,0,...,0,1,0,0,0,0,0,0,0,0
699,lecture,Very Helpful and knowledgeable,NaN,HELP,CON,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,1,0
700,lecture,very helpful and knowledgeable,NaN,HELP,CON,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,1,0
701,lecture,very helpful and knowldgeable,NaN,HELP,CON,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
def set_random_seed(seed=0):
  np.random.seed(seed)
  random.seed(seed)
  #tf.random.set_seed(seed)

In [7]:
from sklearn import model_selection, naive_bayes, svm, linear_model, ensemble, neighbors
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer

In [8]:
def cohens_kappa(tp,fp,fn,tn):
    num = 2* (tp*tn - fn*fp)
    denom = (tp+fp)*(fp+tn) + (tp+fn)*(fn+tn)
    return num/denom

In [21]:
#word pre-processing here
def preprocess_text(para):
    # Remove punctuations and numbers
    para = re.sub('[^a-zA-Z]', ' ', para)
    # Single character removal
    para = re.sub(r"\s+[a-zA-Z]\s+", ' ', para)
    # Removing multiple spaces
    para = re.sub(r'\s+', ' ', para)
    
    tokens = para.split()
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    
    return tokens

Xt = []
for response in df["explanation"].tolist():
    Xt.append(preprocess_text(response))
    
y = np.array(df["ANA"].tolist())

In [22]:
def make_split(s):
    set_random_seed(seed=s)
    Train_X, Test_X, Train_y, Test_y = model_selection.train_test_split(Xt,y,test_size=0.1)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(Train_X)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(Train_X, mode="binary")
    # encode training data set
    Xtest = tokenizer.texts_to_matrix(Test_X, mode="binary")
    return tokenizer, Xtrain, Xtest, Train_y, Test_y

In [23]:
def naivebayes1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    # fit the training dataset on the NB classifier
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_NB = Naive.predict(Xtest)
    # Use accuracy_score function to get the accuracy
    #print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_y)*100)
    #print("Naive Bayes Precision Score -> ",precision_score(predictions_NB, Test_y)*100)
    #print("Naive Bayes Recall Score -> ",recall_score(predictions_NB, Test_y)*100)
    # Confusion Matrix
    #print("Naive Bayes Confusion Matrix -> ",confusion_matrix(predictions_NB, Test_y))
    tn, fp, fn, tp = confusion_matrix(predictions_NB, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_NB, Test_y),precision_score(predictions_NB, Test_y),recall_score(predictions_NB, Test_y),k

In [24]:
def svm1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(Xtest)
    tn, fp, fn, tp = confusion_matrix(predictions_SVM, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_SVM, Test_y),precision_score(predictions_SVM, Test_y),recall_score(predictions_SVM, Test_y),k

In [25]:
def logreg1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    Log = linear_model.LogisticRegression(random_state = s, max_iter = 10000)
    Log.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_Log = Log.predict(Xtest)
    # Use accuracy_score function to get the accuracy
    #print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_Log, Test_y)*100)
    # Confusion Matrix
    #print("Logistic Regression Confusion Matrix -> ",confusion_matrix(predictions_Log, Test_y))
    tn, fp, fn, tp = confusion_matrix(predictions_Log, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_Log, Test_y),precision_score(predictions_Log, Test_y),recall_score(predictions_Log, Test_y),k

In [26]:
def randFor1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    RF = ensemble.RandomForestClassifier(n_estimators =400,criterion="entropy",random_state =s)
    RF.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_RF = RF.predict(Xtest)
    tn, fp, fn, tp = confusion_matrix(predictions_RF, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_RF, Test_y),precision_score(predictions_RF, Test_y),recall_score(predictions_RF, Test_y),k

In [27]:
def KNN1(s,tokenizer, Xtrain, Xtest, Train_y, Test_y):
    set_random_seed(seed = s)
    kNN = neighbors.KNeighborsClassifier(n_neighbors = 3, p = 1)
    kNN.fit(Xtrain,Train_y)
    # predict the labels on validation dataset
    predictions_kNN = kNN.predict(Xtest)
    tn, fp, fn, tp = confusion_matrix(predictions_kNN, Test_y).ravel()
    k = cohens_kappa(tp,fp,fn,tn)
    return accuracy_score(predictions_kNN, Test_y),precision_score(predictions_kNN, Test_y),recall_score(predictions_kNN, Test_y),k

In [28]:
acc = []
prec = []
rec = []
kap = []
for seed in range(40):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = svm1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("SVM")
print("Accuracy:")
print(np.mean(acc))
print(np.std(acc))
print("Precision:")
print(np.mean(prec))
print(np.std(prec))
print("Recall:")
print(np.mean(rec))
print(np.std(rec))
print("Kappa:")
print(np.mean(kap))
print(np.std(kap))

SVM
Accuracy:
0.9306930693069306
0.021235258009432864
Precision:
0.7204469264395734
0.11282704908783875
Recall:
0.7878339869409916
0.10399909170507902
Kappa:
0.7059600582153376
0.09081095798550105


In [29]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = logreg1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("Logistic Regression")
print("Accuracy:")
print(np.mean(acc))
print(np.std(acc))
print("Precision:")
print(np.mean(prec))
print(np.std(prec))
print("Recall:")
print(np.mean(rec))
print(np.std(rec))
print("Kappa:")
print(np.mean(kap))
print(np.std(kap))

Logistic Regression
Accuracy:
0.9386138613861386
0.02376237623762374
Precision:
0.6031111943611943
0.13795512869445656
Recall:
0.9277561327561328
0.07828176796882683
Kappa:
0.6892857129011503
0.11728674040719023


In [30]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = randFor1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("Random Forest")
print("Accuracy:")
print(np.mean(acc))
print(np.std(acc))
print("Precision:")
print(np.mean(prec))
print(np.std(prec))
print("Recall:")
print(np.mean(rec))
print(np.std(rec))
print("Kappa:")
print(np.mean(kap))
print(np.std(kap))

Random Forest
Accuracy:
0.9336633663366335
0.02406979362602616
Precision:
0.5811957486957486
0.13790329516493025
Recall:
0.8950919913419911
0.0941821189247829
Kappa:
0.6615060793716765
0.12524926533131273


In [31]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = KNN1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("k Nearest neighbors")
print(np.mean(acc))
print(np.std(acc))
print(np.mean(prec))
print(np.std(prec))
print(np.mean(rec))
print(np.std(rec))
print(np.mean(kap))
print(np.std(kap))
#svm1, logreg1, randFor1, KNN1, neural_net

k Nearest neighbors
0.8747524752475249
0.026983604108735938
0.3215037046287047
0.1218375310478276
0.5947691197691197
0.16264053796702152
0.3402165421438634
0.11328814244286568


In [32]:
acc = []
prec = []
rec = []
kap = []
for seed in range(20):
    tokenizer, Xtrain, Xtest, Train_y, Test_y = make_split(seed)
    a,p,r,k = naivebayes1(seed,tokenizer, Xtrain, Xtest, Train_y, Test_y)
    acc.append(a)
    prec.append(p)
    rec.append(r)
    kap.append(k)
print("Naive Bayes")
print(np.mean(acc))
print(np.std(acc))
print(np.mean(prec))
print(np.std(prec))
print(np.mean(rec))
print(np.std(rec))
print(np.mean(kap))
print(np.std(kap))
#svm1, logreg1, randFor1, KNN1, neural_net

Naive Bayes
0.9143564356435645
0.022200086460176873
0.5741396103896104
0.12888349170268948
0.7464858058608058
0.09649696926779397
0.5923215914659752
0.10125936561133451
